In [ ]:
from copy import deepcopy
import time
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json
from activity_detector import ActivityDetector, LateralActivityHost, LateralActivityTarget, \
    LeadVehicle, ActivityDetectorParameters, LineData, LateralStateTarget, LongitudinalStateTarget, \
    LongitudinalActivity
from mark_highway import HighwayMarker
from find_index_from_video import approx_index
from ngram import NGram
from ngram_search import find_sequence
import seaborn as sns
from tqdm import tqdm
from typing import Iterable, List, NamedTuple, Tuple
from target_gluer import merge_targets, _TargetGluer, _TargetGluerOptions
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Load the data
i_file = 0
datafiles = glob(os.path.join("data", "1_hdf5", '*.hdf5'))
AD = ActivityDetector(datafiles[i_file])

In [ ]:
AD.set_lon_activities_host()

In [ ]:
AD.set_lat_activities_host()

The following codes checks the recall of the lane changed detection. It shows the activity at the index at which a lane change occurs. These indices are obtained after looking at the video. 

It shows that all left lane changes are detected. Two right lane changes are missed. 

In [ ]:
if i_file == 0:
    # Jeroen's code does not detect the lane change at 1963.60.
    i_left_lane_change = [203.35, 217.29, 1738.83, 1813.3, 1900.71, 1963.60,
                          2000.3, 2171.98, 2310.65, 2318.23, 2546.01, 2650.02]
    print([AD.get("host_lateral_activity", i) for i in i_left_lane_change])
    # Jeroen's code does not detect the lane change at 2193.44 and 2236.72.
    # These lane changes are also difficult to see by eye, so these indices
    # are approximate.
    i_right_lane_change = [251.46, 304.95, 1846.39, 1858.69, 1927.05, 2040.03,
                           2193.44, 2236.72, 2487.13, 2568.65, 2668.15, 2675.00]
    print([AD.get("host_lateral_activity", i) for i in i_right_lane_change])

The following code shows the recall of the cut-in detection at the highway. There are six cut-ins identified from the videos. The lateral activities for these six target vehicles are determined. 
All cut-ins are detected. For some strange reason, the lane changes are detected 3 seconds before the timing that is obtained from the video. That might be a syncing error?

In [ ]:
if i_file == 0:
    # Cut-in targets:
    # 3850: Motorbike from right (29:47, index=1802)
    # 4171: Dark passenger car from right (quite far) (32:50, index=1985)
    # 5028: Silver passenger car from right (39:18, index=2373)
    # 5191: Black van from left (40:53, index=2468)
    # 5645: Black passenger car from left (42:26, index=2562)
    # 6008: Gray VW from left (44:26, index=2681)
    for i_target, index in zip([3850, 4171, 5028, 5191, 5645, 6008],
                               [1802., 1985., 2373., 2468., 2562., 2681.]):
        AD.set_target_activities(i_target)
        print("Activity of target {:d} at index {:.2f}: {:s}".
              format(i_target, index-3, AD.targets[i_target].at[index-3, "lateral_activity"]))

Compute all events for the targets. This takes quite some time, around 13 minutes on the zBook laptop.

In [ ]:
# This takes round 13 minutes!
AD.set_target_activities()

Set the tags for the longitudinal and lateral state of each target vehicle at each time instant. Furthermore, tags are added that specify whether a target vehicle is a lead vehicle at a certain time instant. The states of the targets (i.e., `set_states_targets()`) can only be done after the lateral events (through `set_target_activities()`), because the lateral position of the targets with respect to the lane lines is needed. Similarly, the determination of the lead vehicle (`set_lead_vehicle()`) can only be done after the other states (`set_states_targets()`), because it makes use of the result of the previous step.

`set_states_targets()` is really fast, `set_lead_vehicle()` takes around 3 minutes on the zBook laptop.

In [ ]:
AD.set_states_targets()
AD.set_lead_vehicle()

Determine whether the ego vehicle drives on the highway.

In [ ]:
highway_marker = HighwayMarker(AD.data)
highway_marker.mark_highway()

Save the data!

In [ ]:
AD.to_hdf(datafiles[i_file])

## Create or load n-grams for target vehicles

In [ ]:
filename = os.path.join("data", "4_ngrams", "{:s}.hdf5".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["longitudinal_activity", "lateral_activity", "longitudinal_state", 
              "lateral_state", "lead_vehicle", "id"]
metadata = (("tstart", float), ("tend", float), ("target_id", int))
target_ngrams = NGram(fieldnames, metadata)

In [ ]:
if True or not target_ngrams.from_hdf(filename, "targets"):
    for target in AD.targets:
        target_ngrams.ngram_from_data(target, 
                                      tstart=target.index[0],
                                      tend=target.index[-1],
                                      target_id=int(target["id"].values[0]))
    target_ngrams.sort_ngrams("tstart")
    target_ngrams.to_hdf(filename, "targets", mode="w")

## Create or load n-gram for ego vehicle

In [ ]:
fieldnames = ["host_longitudinal_activity", "host_lateral_activity", "is_highway"]
metadata = (("tstart", float), ("tend", float))
ego_ngram = NGram(fieldnames, metadata)

In [ ]:
if True or not ego_ngram.from_hdf(filename, "ego"):
    ego_ngram.ngram_from_data(AD.data, tstart=AD.data.index[0], tend=AD.data.index[-1])
    ego_ngram.to_hdf(filename, "ego")

## Extract cut-in scenario

For target vehicle, we need:
1. Lateral activity `li` or `ri`.
2. Lateral activity `fl` and lead vehicle `y`.

For the ego vehicle, we need it to go straight during both steps of the target vehicle. Furthermore, the scenario need to take place at the highway.

In [ ]:
target_tags = [dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value],
                    lead_vehicle=[LeadVehicle.NOLEAD.value]),
               dict(lead_vehicle=[LeadVehicle.LEAD.value])]
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True])]

In [ ]:
# Cut-in targets for first dataset:
# 3844: Motorbike from right (29:47, index=1799.29)
# 4114: Black SUV from left (32:28, index=1960.68)
# 4160: Dark passenger car from right (quite far) (32:50, index=1982.01)
# 5014: Silver passenger car from right (39:18, index=2370.28)
# 5177: Black van from left (40:53, index=2465.04)
# 5630: Black passenger car from left (42:26, index=2558.85)
# 5993: Gray VW from left (44:26, index=2677.92)
# File 2: [2430, 2437, 3365, 3478, 3767, (unknown), 4123, 4137]
# File 3: [4577, 5390, 5412, 5449, 5564]
# File 4: [727, (unknown), 3047, (unknown), 3085, 3458, 4224, 4255, 4299, 4550, 5056]
e = ego_ngram.ngram
cutins = []
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        cutins.append((i, search.t_start, search.t_end))
        print(i, search)

Show the data at a cut in.

In [ ]:
maxy = 4
targets = pd.concat(AD.targets)
targets = targets.loc[np.logical_and(targets['dx'] > 0, targets['dx'] < 80)]
targets = targets.loc[np.logical_and(targets['dy'] > -maxy, targets['dy'] < maxy)]
ids = [target["id"].values[0] for target in AD.targets]

In [ ]:
def plot_cutin():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    targets_update = targets.loc[np.logical_and(targets.index >= t_start, targets.index <= t_end)]
    plt.plot([t_start, t_end], [0, 0], 'k-')
    for target_id, target in targets_update.groupby("id"):
        plt.plot(target["line_left_next"], 'b-')
        plt.plot(target["line_right_next"], 'r-')
        
        # Try to find cut-in
        potential = np.logical_or(np.logical_and(target["line_left_next"] > 0,
                                                 target["line_left_prev"] < 0),
                                  np.logical_and(target["line_right_next"] < 0,
                                                 target["line_right_prev"] > 0))
        if np.any(potential):
            print("Potential cut-in of target ID {:.0f} (=target {:d}) at: "
                  .format(target_id, ids.index(target_id)), end="")
            i = target.index[potential].values[0]
            print("{:.2f} (dx={:.1f}, thw={:.2f})"
                  .format(i, target.loc[i, "dx"], target.loc[i, "dx"]/AD.data.loc[i, "Host_vx"]))
    plt.xlim(t_start, t_end)
    plt.ylim(-maxy, maxy)

In [ ]:
minutes = 34
seconds = 0
window = 5
plot_cutin()

In [ ]:
i_target = 3767
target_ngrams.ngrams[i_target]

In [ ]:
ego_ngram.ngram[target_ngrams.metadata["tstart"].values[i_target]-10:
                target_ngrams.metadata["tend"].values[i_target]]

In [ ]:
find_sequence((target_ngrams.ngrams[i_target], ego_ngram.ngram), 
              (target_tags, ego_tags), verbose=2)

In [ ]:
[print(event) for event in AD.lat_activities_target_i(i_target)]
AD.set_target_activities(i_target)
plt.plot(AD.targets[i_target]["line_left_next"], 'b')
plt.plot(AD.targets[i_target]["line_right_next"], 'r')
plt.plot([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]], [0, 0], 'k')
plt.plot(AD.targets[i_target]["line_left_down"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_left_up"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_right_down"], color=(.5, .5, 1))
plt.plot(AD.targets[i_target]["line_right_up"], color=(.5, .5, 1))
_ = plt.xlim([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]])

In [ ]:
i = 2430.48
AD.targets[i_target].loc[i, "dx"] / AD.data.loc[i, "Host_vx"]

In [ ]:
[print(event) for event in AD.lat_activities_target_i(i_target)]
#AD.set_target_activities(i_target)
plt.plot(AD.targets[i_target]["line_left_next"], 'b')
plt.plot(AD.targets[i_target]["line_right_next"], 'r')
plt.plot([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]], [0, 0], 'k')
plt.plot(AD.targets[i_target]["line_left_down"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_left_up"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_right_down"], color=(.5, .5, 1))
plt.plot(AD.targets[i_target]["line_right_up"], color=(.5, .5, 1))
_ = plt.xlim([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]])

## Extract overtaking before changing lane

In [ ]:
target_tags = [dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.REAR.value]),
               dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.FRONT.value]),
               dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.FRONT.value])]
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LEFT_LANE_CHANGE.value],
                 is_highway=[True])]

In [ ]:
e = ego_ngram.ngram
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        print(i, search)

In [ ]:
maxy = 5
targets = pd.concat(AD.targets)
targets = targets.loc[np.abs(targets['dx']) < 100]
targets = targets.loc[np.logical_and(targets['dy'] > -1, targets['dy'] < maxy)]
ids = [target["id"].values[0] for target in AD.targets]

In [ ]:
def plot_overtaking():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    targets_update = targets.loc[np.logical_and(targets.index >= t_start, targets.index <= t_end)]
    for target_id, target in targets_update.groupby("id"):
        plt.plot(-target["dy"], target["dx"], '.')
        if np.any(target["lead_vehicle"] == "y"):
            print("Target {:d} is lead vehicle from {:.2f}"
                  .format(ids.index(target_id), 
                          target.index[np.where(target["lead_vehicle"] == "y")[0][0]]))

In [ ]:
minutes = 42
seconds = 12
window = 10
plot_overtaking()

In [ ]:
i_target = 4181
target_ngrams.ngrams[i_target]

In [ ]:
for i in range(i_target-1, i_target-100, -1):
    if not np.all(AD.targets[i]["dx"] < 0):
        continue
    if AD.targets[i].index[-1] >= AD.targets[i_target].index[0]:
        continue
    if AD.targets[i]["dy"].values[-1] < 0:
        continue
    if AD.targets[i]["dx"].values[-1] < -10:
        continue
    if abs(AD.targets[i]["vx"].values[-1] - AD.targets[i_target]["vx"].values[0]) > 5:
        continue
    v_target = (AD.targets[i]["vx"].values[-1] + AD.targets[i_target]["vx"].values[0])/2
    v_host = AD.data.loc[AD.targets[i].index[-1]:AD.targets[i_target].index[0], "Host_vx"]
    t_gone = AD.targets[i_target].index[0] - AD.targets[i].index[-1]
    x_diff = (AD.targets[i]["dx"].values[-1] - AD.targets[i_target]["dx"].values[0] +
              v_target*t_gone - np.trapz(v_host)*0.01)
    print("{:4d}, dx={:.1f}".format(i, x_diff))

In [ ]:
a = deepcopy(AD)
t = _TargetGluer(a.data, a.targets, 0.01, _TargetGluerOptions(x_max_visible=25))

In [ ]:
i = 4161
info = t.target_in_blind_spot(AD.targets[i])
info

In [ ]:
for j in [i, i_target]:
    plt.plot(-AD.targets[j]["dy"], AD.targets[j]["dx"], '.')

In [ ]:
target = AD.targets[i_target]
print(target.index[-1] - target.index[0] < t.options.t_min_available)
vx_target = np.mean(target["vx"][target.index[-1] - t.options.tsec_v_avg:])
vx_host = t.host.at[target.index[-1], "Host_vx"]
print(abs(vx_target - vx_host) <= t.options.minimum_speed)

In [ ]:
candidate = AD.targets[3874]

In [ ]:
t_gone = candidate.index[0] - info.last_t
x_candidate = candidate["dx"].values[0]
x_absoffset = abs(info.last_x + t_gone * (info.vx_target - info.vx_host) - x_candidate)

In [ ]:
abs(t_gone*info.vx_target + info.last_x - x_candidate -
    np.trapz(AD.data.loc[info.last_t:candidate.index[0], "Host_vx"])*0.01)

In [ ]:
ids = [int(target["id"].iat[0]) for target in AD.targets]

In [ ]:
ids.index(5156)

In [ ]:
for i in [3857, 3871]:
    plt.plot(-AD.targets[i]["dy"], AD.targets[i]["dx"])

In [ ]:
for i in [3854]:
    plt.plot(-AD.targets[i]["dy"], AD.targets[i]["dx"])

## Extract lead braking that causes braking

In [ ]:
target_tags = [dict(lead_vehicle=LeadVehicle.LEAD.value,
                    longitudinal_activity=LongitudinalActivity.DECELERATING.value)]
ego_tags = [dict(is_highway=[True])]

In [ ]:
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        # Find maximum deceleration.
        acceleration = AD.targets[i].loc[search.t_start:search.t_end, "ax"]
        max_deceleration = acceleration.min()
        print(i, search, "Maximum deceleration: {:.2f}".format(-max_deceleration))